<a href="https://colab.research.google.com/github/bo-bits/nn-zero-to-hero/blob/master/exercises/makemore_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

In [303]:
# Trigram Model

words = open('names.txt', 'r').read().splitlines()


In [335]:
# Initialize dictionaries stoi itos

chars = sorted(list(set(''.join(words))))

chars.insert(0, '.')  # Insert the period as the first character

# Initialize stoi for single characters
stoi = {char: i for i, char in enumerate(chars)}  # Single character to index mapping

# Start bigram indices after the single characters
bigram_start_idx = len(stoi)

# Add bigrams to stoi with unique indices
for i, (char1, char2) in enumerate((a + b for a in chars for b in chars), bigram_start_idx):
    stoi[char1 + char2] = i

# Create itos for reverse mapping

stoi

{'.': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26,
 '..': 27,
 '.a': 28,
 '.b': 29,
 '.c': 30,
 '.d': 31,
 '.e': 32,
 '.f': 33,
 '.g': 34,
 '.h': 35,
 '.i': 36,
 '.j': 37,
 '.k': 38,
 '.l': 39,
 '.m': 40,
 '.n': 41,
 '.o': 42,
 '.p': 43,
 '.q': 44,
 '.r': 45,
 '.s': 46,
 '.t': 47,
 '.u': 48,
 '.v': 49,
 '.w': 50,
 '.x': 51,
 '.y': 52,
 '.z': 53,
 'a.': 54,
 'aa': 55,
 'ab': 56,
 'ac': 57,
 'ad': 58,
 'ae': 59,
 'af': 60,
 'ag': 61,
 'ah': 62,
 'ai': 63,
 'aj': 64,
 'ak': 65,
 'al': 66,
 'am': 67,
 'an': 68,
 'ao': 69,
 'ap': 70,
 'aq': 71,
 'ar': 72,
 'as': 73,
 'at': 74,
 'au': 75,
 'av': 76,
 'aw': 77,
 'ax': 78,
 'ay': 79,
 'az': 80,
 'b.': 81,
 'ba': 82,
 'bb': 83,
 'bc': 84,
 'bd': 85,
 'be': 86,
 'bf': 87,
 'bg': 88,
 'bh': 89,
 'bi': 90,
 'bj': 91,
 'bk': 92,
 'bl': 93,
 'b

In [ ]:
import torch
import torch.nn.functional as F

# Initialize
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((756, 27), generator=g, requires_grad=True)

# create the training set of trigrams (x,y)
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1+ch2]
    ix3 = stoi[ch3]
    xs.append(ix1)
    ys.append(ix3)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

In [332]:
for k in range(100):
  # forward pass
  xenc = F.one_hot(xs, num_classes=756).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(num), ys].log().mean()
  print(f"Loss: ",loss.item())

  # backward pass
  # Calculated by pytoch by computing gradients
  W.grad = None # set to zero the gradient
  loss.backward()
  W.data += -50 * W.grad

Loss:  3.786095142364502
Loss:  3.4637839794158936
Loss:  3.1635053157806396
Loss:  2.8889341354370117
Loss:  2.6438236236572266
Loss:  2.429399251937866
Loss:  2.2430198192596436
Loss:  2.080261468887329
Loss:  1.9371535778045654
Loss:  1.8106318712234497
Loss:  1.6983482837677002
Loss:  1.5984528064727783
Loss:  1.509432315826416
Loss:  1.4300153255462646
Loss:  1.359114646911621
Loss:  1.295795202255249
Loss:  1.23924720287323
Loss:  1.1887630224227905
Loss:  1.1437134742736816
Loss:  1.1035327911376953
Loss:  1.0677061080932617
Loss:  1.035763144493103
Loss:  1.007274866104126
Loss:  0.9818510413169861
Loss:  0.9591383934020996
Loss:  0.9388185143470764
Loss:  0.9206076264381409
Loss:  0.9042524695396423
Loss:  0.889529824256897
Loss:  0.8762427568435669
Loss:  0.8642190098762512
Loss:  0.853307843208313
Loss:  0.8433783054351807
Loss:  0.8343161344528198
Loss:  0.8260222673416138
Loss:  0.8184104561805725
Loss:  0.8114060163497925
Loss:  0.8049435019493103
Loss:  0.798966169357299

In [ ]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  out = []
  ix = 0
  while True:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=756).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character

    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

In [169]:
# Calculates the average negative log likelihood for a single example
# what probability the model assigned to the correct output, take the log (high prob -> 0)
# take negative, to make output positive
# negative log likelihood is an indicator of how well the NN performed, we want to minimize
nlls = torch.zeros(4)
for i in range(4):
  # i-th bigram:
  x = xs[i].item() # input character index
  y = ys[i].item() # label character index
  print('--------')
  print(f'bigram example {i+1}: {itob[x]}{itos[y]} (indexes {x},{y})')
  print('input to the neural net:', x)
  print('output probabilities from the neural net:', probs[i])
  print('label (actual next character):', y)
  p = probs[i, y]
  print('probability assigned by the net to the the correct character:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

--------
bigram example 1: .ma (indexes 15,1)
input to the neural net: 15
output probabilities from the neural net: tensor([6.9801e-05, 9.9624e-01, 3.5564e-05, 2.3088e-04, 1.9466e-04, 5.6965e-05,
        2.6365e-04, 4.2325e-04, 9.7273e-05, 7.0569e-05, 2.0174e-05, 3.0514e-04,
        2.3514e-04, 1.6287e-04, 1.1365e-04, 7.4941e-05, 1.5874e-04, 7.9995e-05,
        1.2383e-04, 1.8382e-05, 1.6627e-04, 2.1135e-04, 8.3075e-05, 2.9599e-05,
        6.8066e-05, 1.1100e-04, 3.5287e-04], grad_fn=<SelectBackward0>)
label (actual next character): 1
probability assigned by the net to the the correct character: 0.9962422847747803
log likelihood: -0.003764793276786804
negative log likelihood: 0.003764793276786804
--------
bigram example 2: mar (indexes 367,18)
input to the neural net: 367
output probabilities from the neural net: tensor([1.3363e-05, 9.7108e-05, 1.5531e-04, 9.7857e-05, 1.5648e-04, 1.6445e-04,
        1.0697e-04, 3.3253e-04, 6.3384e-05, 1.1796e-04, 4.3205e-04, 5.4818e-05,
        2.3334e

E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?

In [63]:
# Exercise 2

E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?

In [64]:
# Exercise 1

E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?

E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?


E06: meta-exercise! Think of a fun/interesting exercise and complete it.